In [ ]:
import pandas as pd
import numpy as np

## Data importation and understanding

In [ ]:
movies_df = pd.read_csv('ml-latest-small/movies.csv', sep=',', names=['item_id', 'title', 'genres'], engine='python',skiprows=1)
tags_df = pd.read_csv('ml-latest-small/tags.csv', sep=',', names=['user_id', 'item_id', 'tag', 'timestamp'], engine='python',skiprows=1)
ratings_df = pd.read_csv('ml-latest-small/ratings.csv', sep=',', names=['user_id', 'item_id', 'rating', 'timestamp'], engine='python',skiprows=1)

The MovieLens dataset is a widely-used dataset for movie recommendation systems, collected by the GroupLens research group at the University of Minnesota. It consists of multiple tables containing data about movies, ratings, and tags provided by users. Here’s a description of each table and its variables:

### movies_df
- **item_id**: Unique identifier for movies.
- **title**: The title of the movie, typically containing the release year in parentheses.
- **genres**: A pipe-separated list of genres associated with the movie.

This DataFrame contains metadata about the movies. Each movie is identified by a unique ID and has associated attributes like the title and the list of genres it belongs to. The genres are categorical and are typically used to filter or describe the content of the movie.

In [ ]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   item_id  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


### tags_df
- **user_id**: Unique identifier for users.
- **item_id**: Unique identifier for movies which corresponds to `item_id` in `movies_df`.
- **tag**: Textual tag provided by the user for a movie. These can be descriptive words or short phrases.
- **timestamp**: The timestamp when the tag was provided. This is typically a Unix time stamp and indicates when the user tagged the movie.

The `tags_df` table contains user-generated metadata for the movies. Each row indicates that a particular user has tagged a movie with a textual descriptor. These tags can be used for content-based filtering or to enhance the information about a movie beyond its genres.

In [ ]:
tags_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    3683 non-null   int64 
 1   item_id    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


### ratings_df
- **user_id**: Unique identifier for users, which corresponds to `user_id` in `tags_df`.
- **item_id**: Unique identifier for movies, consistent with `item_id` in both `movies_df` and `tags_df`.
- **rating**: The rating given to a movie by a user. This is typically on a defined scale, like 0.5 to 5 stars.
- **timestamp**: The timestamp when the rating was provided, in the same format as in `tags_df`.

The `ratings_df` table is a record of user ratings for movies. Each row documents that a user has assigned a numerical rating to a movie. This is the core data used in collaborative filtering for recommendation systems.

In [ ]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    100836 non-null  int64  
 1   item_id    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


When using these tables for building a recommendation system, one usually starts with the `ratings_df` to build the utility matrix that relates users to movies through the ratings they've provided. `movies_df` can be used to provide readable movie titles and genres for recommendations, and `tags_df` can add additional context for the movies or for more sophisticated recommendation systems that also use content-based filtering methods.

## Exploring the data

In [ ]:
# For movies_df to count unique movies
unique_movies = movies_df['item_id'].nunique()
print(f'Unique movies: {unique_movies}')

# For ratings_df to count unique users and movies
unique_users_ratings = ratings_df['user_id'].nunique()
unique_movies_ratings = ratings_df['item_id'].nunique()
print(f'Unique users in ratings: {unique_users_ratings}')
print(f'Unique movies in ratings: {unique_movies_ratings}')

# For tags_df to count unique users, movies, and tags
unique_users_tags = tags_df['user_id'].nunique()
unique_movies_tags = tags_df['item_id'].nunique()
unique_tags = tags_df['tag'].nunique()
print(f'Unique users in tags: {unique_users_tags}')
print(f'Unique movies in tags: {unique_movies_tags}')
print(f'Unique tags: {unique_tags}')

Unique movies: 9742
Unique users in ratings: 610
Unique movies in ratings: 9724
Unique users in tags: 58
Unique movies in tags: 1572
Unique tags: 1589


## Small preprocess

We first transform the columns in the table `ratings_df` to *int32*, as the recommender model needs this adaptation

In [ ]:
ratings_df['user_id']=ratings_df['user_id'].astype('int32')
ratings_df['item_id']=ratings_df['item_id'].astype('int32')
ratings_df['timestamp']=ratings_df['timestamp'].astype('int32') #It is required by the model

We split the title of the movies in the `movies_df`, creating another column for the year of the film.

After that, we will get rid of the characters ', The', which are at the last part of some titles.

In [ ]:
movies_df['year'] = movies_df['title'].str.extract(r'\((\d{4}(?:–\d{4})?)\)')  # Extracting the year or year range into a new column
movies_df['year'] = movies_df['year'].where(movies_df['year'].str.len() == 4, None)  # Set to None if not a single year
movies_df['title'] = movies_df['title'].str.replace(r'\(\d{4}(?:–\d{4})?\)', '', regex=True).str.strip()  # Removing the year from the title

movies_df['title']=movies_df['title'].str.replace(', The', '')
movies_df['genres']=movies_df['genres'].str.replace('|',', ')

In [ ]:
from imdb import IMDb

na_rows = movies_df['year'].isna()

ia = IMDb()

def get_movie_year(title):
    try:
        movie = ia.search_movie(title)[0]
        return movie['year']
    except Exception as e:
        print(f"Error fetching year for '{title}': {e}")
        return None

# Apply the function to the dataframe
movies_df.loc[na_rows,'year'] = movies_df.loc[na_rows,'title'].apply(get_movie_year)

movies_df['year'] = movies_df['year'].astype(int)
print(movies_df)

      item_id                               title  \
0           1                           Toy Story   
1           2                             Jumanji   
2           3                    Grumpier Old Men   
3           4                   Waiting to Exhale   
4           5         Father of the Bride Part II   
...       ...                                 ...   
9737   193581  Black Butler: Book of the Atlantic   
9738   193583               No Game No Life: Zero   
9739   193585                               Flint   
9740   193587        Bungo Stray Dogs: Dead Apple   
9741   193609        Andrew Dice Clay: Dice Rules   

                                               genres  year  
0     Adventure, Animation, Children, Comedy, Fantasy  1995  
1                        Adventure, Children, Fantasy  1995  
2                                     Comedy, Romance  1995  
3                              Comedy, Drama, Romance  1995  
4                                              Comedy

## Implementing the recommender

Here's a brief explanation of some common hyperparameters for factorization models and a suggested grid for searching:

1. **n_iter**: The number of epochs to run when training the model. More epochs could lead to better performance but also increase the risk of overfitting and computational time.
   - Suggested grid: `[1, 5, 10, 20]`

2. **embedding_dim**: The size of the latent feature vectors for users and items. Larger dimensions could capture more complex patterns but might overfit.
   - Suggested grid: `[8, 16, 32, 64]`

3. **learning_rate**: The step size at each iteration of the optimization algorithm. A smaller learning rate could lead to more precise convergence but might require more epochs.
   - Suggested grid: `[0.001, 0.01, 0.1]`

4. **l2**: The L2 regularization penalty. Higher values could prevent overfitting but might lead to underfitting if too large.
   - Suggested grid: `[0.0, 1e-6, 1e-5, 1e-3]`

5. **loss**: The loss function to be used. Common options are `pointwise`, `bpr`, or `hinge` loss, which are suitable for different types of recommendation tasks.
   - Suggested values: `['pointwise', 'bpr', 'hinge']`

6. **batch_size**: The number of samples per gradient update. Larger batches provide more accurate estimates of the gradient, but smaller batches might help the model to generalize better.
   - Suggested grid: `[32, 64, 128, 256]`


### Training the model

In [ ]:
from spotlight.interactions import Interactions
from spotlight.factorization.explicit import ExplicitFactorizationModel
from spotlight.cross_validation import random_train_test_split

# Step 2: Create the Interactions object
interaction_data = Interactions(
    user_ids=ratings_df['user_id'].values,
    item_ids=ratings_df['item_id'].values,
    ratings=ratings_df['rating'].values,
    timestamps=ratings_df['timestamp'].values
)

# Split the interaction data into training and test sets
train, test = random_train_test_split(interaction_data)

# Initialize the model
model = ExplicitFactorizationModel(
    n_iter=5,           # Number of epochs of training
    embedding_dim=32,   # Latent factors (embedding size)
    use_cuda=False      # If you have a CUDA capable GPU, set to True to speed up training
)

# Fit the model on the training data
model.fit(train, verbose=True)

Epoch 0: loss 4.224278883362341
Epoch 1: loss 0.8214049972551845
Epoch 2: loss 0.5264862054814858
Epoch 3: loss 0.3726893660714355
Epoch 4: loss 0.29536568744777913


In [ ]:
from joblib import dump, load

#dump(model, 'model.pkl')

model = load('model.pkl')


In [ ]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(model, train)
test_rmse = rmse_score(model, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

### R1: Community recommender

Given a tag, a list of popular tags and a number of films (n), it returns the most *n* recommended films (ordered by rating) in the tag provided.

If the tag is not in the popular tags list, it returns an error.

In [ ]:
# We first filter popular tags, which have more than 15 users
counter_tag= tags_df['tag'].value_counts()
popular_tag = counter_tag[counter_tag > 15]
popular_tag.head()

tag
In Netflix queue     131
atmospheric           36
thought-provoking     24
superhero             24
funny                 23
Name: count, dtype: int64

In [ ]:
def community_recommender(tag, popular_tag, n, genre=None):
    popular_tag = [t.lower() for t in popular_tag.index] 
    tag = tag.lower()  # convert everything to lower case
    if tag in popular_tag:
        movie_filtered_by_tag = tags_df[tags_df['tag'].str.lower() == tag]
        movie_list = movie_filtered_by_tag['item_id']  # we extract here the movies that have this tag
        movies_filtered = ratings_df[ratings_df['item_id'].isin(movie_list)].groupby(['item_id'])['rating'].mean().reset_index()  # here we focus on those movies and then we extract the mean rating.
        
        # If genre filtering is enabled, filter the movies_df first
        if genre:
            movies_df_filtered_by_genre = movies_df[movies_df['genres'].str.contains(genre, case=False, na=False)]
            movies_filtered = movies_filtered[movies_filtered['item_id'].isin(movies_df_filtered_by_genre['item_id'])]
        
        movies_filtered.sort_values(by='rating', ascending=False, inplace=True)  # we sort them 
        top_movie_ids = movies_filtered.head(n)['item_id']  # here we extract the first n movies
        recommended_movies = movies_df[movies_df['item_id'].isin(top_movie_ids)]['title'].values.tolist()  # we extract the movies with that id
        return recommended_movies
    else:
        return 'No matches based on your research'

In [ ]:
print("\n".join(community_recommender('suspense',popular_tag,10, genre='Comedy'))) #Example of how to use it.

Pulp Fiction


### R2: Rating prediction given a user and a movie

Given a user_id and a movie_title, it predicts the rating for the given movie and user.

In [ ]:
def predict_rating_for_user_and_movie(model, user_id, movie_title, movies_df):
    # Predict scores for all items for the user
    pred = model.predict(user_ids=np.array([user_id]))

    # Get the movie indices sorted by predicted rating
    sorted_indices = np.argsort(pred)[::-1]

    # Find the movie ID corresponding to the given movie title
    movie_row = movies_df[movies_df['title'] == movie_title]
    if movie_row.empty:
        return "Movie title not found."

    movie_id = movie_row['item_id'].iloc[0]

    # Get the sorted index for this movie ID
    sorted_index = np.where(sorted_indices == movie_id)[0][0]

    # Get the predicted rating for the movie
    predicted_rating = pred[sorted_indices[sorted_index]]

    return predicted_rating

# Example usage
user_id = 1  # replace with the user ID
movie_title = "Silence of the Lambs"  # replace with the movie title
predicted_rating = predict_rating_for_user_and_movie(model, user_id, movie_title, movies_df)
print(f"Predicted rating for user {user_id} and movie '{movie_title}': {predicted_rating}")


Predicted rating for user 1 and movie 'Silence of the Lambs': 4.287411689758301


### R3: N recommended movies given a user

Given a user_id and number of recommendations (n), it returns the *n* recommended movies for that user.
We also filter by year and genre

In [ ]:
def recommended_movies_by_user(model, user_id, n_movies, movies_df, genres=None, start_year=None, end_year=None):
    # Apply genre/year filters before making predictions
    if genres is not None:
        filter_condition = lambda x: any(genre.lower() in x.lower() for genre in genres)
        movies_df = movies_df[movies_df['genres'].apply(filter_condition)]
    if start_year is not None:
        if end_year is not None:
            movies_df = movies_df[(movies_df['year'] >= start_year) & (movies_df['year'] <= end_year)]
        else:
            movies_df = movies_df[movies_df['year'] >= start_year]

    # Now predict scores for the (optionally) filtered items for the user
    filtered_movie_ids = movies_df['item_id'].values
    pred = model.predict(user_ids=np.array([user_id]*len(filtered_movie_ids)), item_ids=filtered_movie_ids)

    # Sort predicted ratings in descending order
    sorted_indices = np.argsort(pred)[::-1]
    #print(len(sorted_indices))
    # Select the top n_movies
    top_indices = sorted_indices[:n_movies]
    recommended_movies = movies_df.iloc[top_indices]['title'].tolist()
    predicted_ratings = pred[top_indices]

    return recommended_movies, predicted_ratings

We have the following unique genres

In [ ]:
unique_genres=['Action', 'Adventure', 'Animation',
       'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western'] 

In [ ]:
min_year=movies_df['year'].min()
max_year=movies_df['year'].max()
print(min_year,max_year)

1902 2018


In [ ]:
years=np.unique(movies_df['year'])

In [ ]:
recommended_movies_by_user(model=model, user_id=1, n_movies=10, genres=['Action','Adventure'],movies_df=movies_df, start_year=2010)

(['Heat',
  'Sherlock Holmes: A Game of Shadows',
  'Day of the Doctor',
  'Jack Reacher',
  'The Man from U.N.C.L.E.',
  'Star Trek Beyond',
  'Knight and Day',
  'Black Panther',
  'Other Guys',
  'Dredd'],
 array([5.7866364, 5.7554164, 5.6918683, 5.5916624, 5.5496798, 5.520312 ,
        5.5181866, 5.4266315, 5.3701644, 5.367329 ], dtype=float32))

### R4: N recommended movies for unseen users

Given a list of movies the user has seen and like, it returns the *n* movies most recommended, similar to the movies the user pass.

For that, we need another model which takes as input a list of movies, rather than a user_id.

In [ ]:
from spotlight.sequence.implicit import ImplicitSequenceModel
from spotlight.cross_validation import user_based_train_test_split

train, test = user_based_train_test_split(interaction_data)

train = train.to_sequence()
test = test.to_sequence()
model_seq = ImplicitSequenceModel(n_iter=5,
                                  representation='cnn',
                                  loss='bpr')
model_seq.fit(train)

In [ ]:
def recommend_next_movies(list_of_movies, model, n_movies, genre=None):
    # We first check if the titles are in the movie list and remove the ones that are not present.
    # We also convert the titles in order to avoid case sensitivity (e.g., 'Spiderman' vs. 'spiderman').
    list_of_movies = [title.lower() for title in list_of_movies]
    list_of_movies = [title for title in list_of_movies if title in movies_df['title'].str.lower().tolist()]
    if not list_of_movies:  # Check if it is empty.
        return 'The list of films provided is not in the catalogue'
    # Get the indices for the movies.
    indices_movies = movies_df[movies_df['title'].str.lower().isin(list_of_movies)]['item_id']
    # Predict the next movies.
    pred = model.predict(sequences=np.array(indices_movies))
    sorted_indices = np.argsort(pred)[::-1]
    
    # Filter out the movies that have already been seen
    sorted_indices = [idx for idx in sorted_indices if idx not in indices_movies.values]
    
    # Apply genre filtering if genre is specified.
    if genre:
        genre_mask = movies_df['genres'].str.contains(genre, case=False, na=False)
        # Use the mask to filter indices
        filtered_indices = genre_mask.index[genre_mask].intersection(sorted_indices)
        # Get the top n_movies indices from the filtered list
        top_indices = filtered_indices[:n_movies]
    else:
        # If no genre filter is applied, simply take the top n_movies indices
        top_indices = sorted_indices[:n_movies]
        
    # Get the recommended movie titles.
    recommended_movies = movies_df[movies_df['item_id'].isin(top_indices)]['title'].values.tolist()

    return recommended_movies


In [ ]:
list_movies = ['Silence of the Lambs', 'Pulp Fiction', 'Flirting With Disaster','Amateur','Scream']

recommend_next_movies(list_movies, model_seq, 5, genre='Comedy')

['Jumanji', 'Grumpier Old Men', 'Waiting to Exhale', 'Heat']

In [ ]:
import pandas as pd
import numpy as np
from spotlight.sequence.implicit import ImplicitSequenceModel
from spotlight.sequence.representations import CNNNet
from spotlight.cross_validation import random_train_test_split
from spotlight.evaluation import sequence_mrr_score

train, test = user_based_train_test_split(interaction_data)

# Convert to sequences suitable for the model
train = train.to_sequence(max_sequence_length=20)
test = test.to_sequence(max_sequence_length=20)

# Define the representation. Here we use a CNN.
representation = CNNNet(train.num_items, embedding_dim=32, kernel_width=3, num_layers=1)

# Create the ImplicitSequenceModel with specified representation
model_seq = ImplicitSequenceModel(n_iter=10,
                                  representation=representation,
                                  loss='bpr',
                                  batch_size=1024,
                                  learning_rate=1e-2,
                                  l2=1e-5,
                                  random_state=np.random.RandomState(42))

# Fit the model
model_seq.fit(train, verbose=True)

# Evaluate the model
mrr = sequence_mrr_score(model_seq, test)
print(f'Mean Reciprocal Rank: {mrr.mean()}')

In [ ]:
def recommend_next_movies(model, movies_df, user_interactions, n_movies=5):
    # Predict scores for all items for a given user's interaction sequence
    user_interaction_sequence = np.array([user_interactions], dtype=np.int32)
    pred = model.predict(user_interaction_sequence)
    
    # Get the movie indices sorted by predicted rating
    sorted_indices = np.argsort(pred[0])[::-1]

    # Filter out movies the user has already seen
    seen_indices = set(user_interactions)
    recommended_indices = [i for i in sorted_indices if i not in seen_indices][:n_movies]
    
    # Get the recommended movie titles using the top indices
    recommended_movies = movies_df.loc[movies_df['item_id'].isin(recommended_indices), 'title'].values.tolist()

    return recommended_movies

# Example usage:
# Assuming you have a user with user_id = 123 and you want to recommend movies
user_id = 123
user_interactions = interactions_df[interactions_df['user_id'] == user_id]['item_id'].tolist()
recommendations = recommend_next_movies(model_seq, movies_df, user_interactions)
print(f'Recommended movies for user {user_id}: {recommendations}')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming interactions_df is your dataset with at least 'item_id' and 'rating'.
# Let's also assume that movies_df is your dataset with at least 'item_id' and 'title'.

# Step 1: Aggregate the ratings
ratings_count = ratings_df.groupby('item_id')['rating'].count().reset_index(name='rating_count')

# Step 2: Merge the movie titles
ratings_with_titles = ratings_count.merge(movies_df[['item_id', 'title']], on='item_id')

# Step 3: Sort the data
sorted_ratings = ratings_with_titles.sort_values(by='rating_count', ascending=False)

# Step 4: Visualization
plt.figure(figsize=(12, 6))
plt.barh(sorted_ratings['title'][:10], sorted_ratings['rating_count'][:10])
plt.xlabel('Number of Ratings')
plt.ylabel('Movies')
plt.title('Top 10 Movies with Most Ratings')
plt.gca().invert_yaxis()  # To display the highest value on top
plt.show()
